In [1]:
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm
# def load_bge(device=None, model_name="bge15small"):
#     if device is None:
#         device = "cuda" if torch.cuda.is_available() else "cpu"
#     print("Using device:", device)

#     # 官方 baseline 建议：推理用 FP32，别 .half()，保证排序稳定
#     model = SentenceTransformer(model_name, device=device, local_files_only=True)
#     model = model.half()
#     model.max_seq_length = 512

#     def encode(texts, batch_size=256):
#         all_emb = []
#         for i in tqdm(range(0, len(texts), batch_size), desc="Encoding"):
#             batch = texts[i:i+batch_size]
#             emb = model.encode(
#                 batch,
#                 convert_to_tensor=True,
#                 batch_size=batch_size,
#                 device=device,
#                 show_progress_bar=False,
#                 normalize_embeddings=True,
#             )
#             all_emb.append(emb)
#         return torch.cat(all_emb, dim=0)

#     # 给自己挂一个 encode_cuda 方便后面调用
#     model.encode_cuda = encode
#     return model




/home/smen/.conda/envs/mtrag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import transformers
transformers.__version__

/home/smen/.conda/envs/mtrag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'4.57.3'

In [1]:
# def load_kingsoft(model_name="Kingsoft-LLM/QZhou-Embedding", device="cuda"):
#     """
#     Kingsoft-QZhou embedding loader with batching and encode_cuda().
#     """

#     print("Using device:", device)

#     # Important: Kingsoft uses trust_remote_code + custom forward
#     model = SentenceTransformer(
#         model_name,
#         device=device,
#         trust_remote_code=True,
#         model_kwargs={
#             "device_map": device,
#             "trust_remote_code": True
#         },
#         tokenizer_kwargs={
#             "padding_side": "left",
#             "trust_remote_code": True
#         }
#     )

#     # FP16 is OK for Kingsoft (not like BGE baseline)
#     model = model.half()

#     # Kingsoft max length = 2048, but 512 is OK for BEIR/MTRAG
#     model.max_seq_length = 512

#     # -----------------------------
#     #        Batch Encoder
#     # -----------------------------
#     def encode(texts, batch_size=64, prompt_name=None):
#         """
#         texts: list[str]
#         prompt_name: "query" or "document"
#         """
#         all_emb = []

#         for i in tqdm(range(0, len(texts), batch_size), desc="Encoding"):
#             batch = texts[i:i + batch_size]

#             emb = model.encode(
#                 batch,
#                 convert_to_tensor=True,
#                 device=device,
#                 show_progress_bar=False,
#                 batch_size=batch_size,
#                 normalize_embeddings=True,
#                 prompt_name=prompt_name,   # KINGSOFT FEATURE
#             )

#             all_emb.append(emb)

#         return torch.cat(all_emb, dim=0)

#     # Attach helper
#     model.encode_cuda = encode
#     return model


In [3]:
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm

def load_qwen3_st(model_name="Qwen3-Embedding-4B", device=None):
    """
    Load Qwen3 embedding model using SentenceTransformers.

    Notes:
      - flash_attention_2 is DISABLED (your GPU cannot use it)
      - SentenceTransformers will:
            • handle query/document prompts
            • handle left padding
            • handle pooling + normalization
      - We only provide efficient batching through encode_cuda()
    """

    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    print("Using device:", device)

    # ---- SAFE FOR HPC ----
    model = SentenceTransformer(
        model_name,
        model_kwargs={
            "device_map": "cuda",      # works on HPC
            "trust_remote_code": True,
            "attn_implementation": None,   # <- IMPORTANT: disable flash-attention2
        },
        tokenizer_kwargs={
            "padding_side": "left",    # Recommended by Qwen team
            "trust_remote_code": True,
        },
        trust_remote_code=True,
        local_files_only=True
        
    )
    model.max_seq_length = 512

    # ------------------------------
    # Custom batch encode wrapper
    # ------------------------------
    def encode(texts, batch_size=32, prompt_name=None):
        """
        Batch embed texts using model.encode() internally.
        prompt_name:
            - None        → document encoding
            - "query"     → query prompt from Qwen config
        """
        all_emb = []

        for i in tqdm(range(0, len(texts), batch_size), desc="Encoding"):
            batch = texts[i:i+batch_size]

            emb = model.encode(
                batch,
                batch_size=batch_size,
                convert_to_tensor=True,
                show_progress_bar=False,
                normalize_embeddings=True,     # IMPORTANT for retrieval
                prompt_name=prompt_name,
            )

            all_emb.append(emb.cpu())

        return torch.cat(all_emb, dim=0)

    # attach method
    model.encode_cuda = encode
    return model


In [4]:
def load_corpus(corpus_path):
    corpus = {}
    with open(corpus_path, "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line)
            doc_id = item["document_id"] if "document_id" in item else item["_id"]
            title = item.get("title", "")
            text = item.get("text", "")
            corpus[doc_id] = {
                "title": title,
                "text": text,
            }
    return corpus

def load_queries(query_path):
    queries = {}
    with open(query_path, "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line)
            qid = item.get("_id") or item.get("query_id") or item["task_id"]
            text = item["text"]
            queries[qid] = text
    return queries


In [ ]:
# def run_retrieval_for_collection(name, cfg, model, top_k=50):
#     root = cfg["root"]
#     corpus_path = os.path.join(root, cfg["corpus_file"])
#     query_path = os.path.join(root, cfg["query_file"])

#     print(f"\n========== Collection: {name} ==========")
#     print("Corpus:", corpus_path)
#     print("Queries:", query_path)

#     corpus = load_corpus(corpus_path)
#     queries = load_queries(query_path)

#     # 1) encode corpus
#     doc_ids = list(corpus.keys())
#     doc_texts = [(corpus[d].get("title", "") + " " + corpus[d].get("text", "")).strip()
#                  for d in doc_ids]
#     # doc_emb = model.encode_cuda(doc_texts, batch_size=8)
#     doc_emb = model.encode_cuda(
#         doc_texts,
#         batch_size=64,
#         prompt_name="document"
#     )

#     print("doc_emb shape:", doc_emb.shape)

#     # 2) encode queries
#     q_ids = list(queries.keys())
#     q_texts = [queries[q] for q in q_ids]
#     # q_emb = model.encode_cuda(q_texts, batch_size=8)
#     q_emb = model.encode_cuda(
#         q_texts,
#         batch_size=64,
#         prompt_name="query"
#     )

#     print("q_emb shape:", q_emb.shape)

#     # 3) brute-force retrieval (cosine = dot，因为已经 normalize)
#     sims = torch.matmul(q_emb, doc_emb.T)   # [num_q, num_doc]

#     results = []  # 用于写 JSONL 的行
#     for qi, qid in enumerate(tqdm(q_ids, desc="Building results")):
#         scores = sims[qi]
#         vals, idx = torch.topk(scores, top_k)
#         ctxs = []
#         for v, ix in zip(vals.tolist(), idx.tolist()):
#             ctxs.append({
#                 "document_id": doc_ids[ix],
#                 "score": float(v),
#             })

#         results.append({
#             "task_id": qid,                  # 对应 run_retrieval_eval 里的 query_id
#             "contexts": ctxs,
#             "Collection": cfg["collection_name"],
#         })

#     return results


In [5]:
def run_retrieval_for_collection(name, cfg, model, top_k=50):
    import os, torch
    from tqdm import tqdm

    root = cfg["root"]
    corpus_path = os.path.join(root, cfg["corpus_file"])
    query_path = os.path.join(root, cfg["query_file"])

    print(f"\n========== Collection: {name} ==========")
    print("Corpus:", corpus_path)
    print("Queries:", query_path)

    # Load data
    corpus = load_corpus(corpus_path)
    queries = load_queries(query_path)

    # ------------------------------
    # Encode documents (NO PROMPT)
    # ------------------------------
    doc_ids = list(corpus.keys())
    doc_texts = [
        (corpus[d].get("title", "") + " " + corpus[d].get("text", "")).strip()
        for d in doc_ids
    ]

    doc_emb = model.encode_cuda(doc_texts, batch_size=128, prompt_name=None)
    print("doc_emb shape:", doc_emb.shape)

    # ------------------------------
    # Encode queries (WITH Qwen3 query prompt)
    # ------------------------------
    q_ids = list(queries.keys())
    q_texts = [queries[q] for q in q_ids]

    q_emb = model.encode_cuda(q_texts, batch_size=128, prompt_name="query")
    print("q_emb shape:", q_emb.shape)

    # ------------------------------
    # Cosine similarity (already normalized)
    # ------------------------------
    sims = torch.matmul(q_emb, doc_emb.T)

    results = []
    for qi, qid in enumerate(tqdm(q_ids, desc="Building results")):
        vals, idx = torch.topk(sims[qi], top_k)
        ctxs = [
            {"document_id": doc_ids[j], "score": float(v)}
            for v, j in zip(vals.tolist(), idx.tolist())
        ]

        results.append({
            "task_id": qid,
            "contexts": ctxs,
            "Collection": cfg["collection_name"],
        })

    return results


In [6]:
def build_submission(submission_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # model = load_bge(model_name=model_name, device=device)
    # model = load_qwen(model_name="Qwen3")
    # model = load_kingsoft(model_name=model_name, device=device)
    model = load_qwen3_st(model_name="Qwen3-Embedding-0.6B", device=device)

    all_results = []
    for name, cfg in COLLECTIONS.items():
        res = run_retrieval_for_collection(name, cfg, model, top_k=50)
        all_results.extend(res)

    print(f"\nTotal tasks: {len(all_results)}")
    with open(submission_path, "w", encoding="utf-8") as f:
        for item in all_results:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")
    print("Saved submission to:", submission_path)
    return submission_path


In [7]:
import subprocess
import os

def run_official_eval(input_file, output_file):
    cmd = [
        "python3",
        "scripts/evaluation/run_retrieval_eval.py",
        "--input_file", input_file,
        "--output_file", output_file,
        "--model_name", model_name,
        "--task_name", task_name
    ]
    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True)
    print("Done, scored file:", output_file)


In [ ]:
import gc
# del model
gc.collect()


torch.cuda.empty_cache()


In [ ]:

import os, json, torch
from tqdm import tqdm

COLLECTIONS = {
    "clapnq": {
        "collection_name": "mt-rag-clapnq-elser-512-100-20240503",
        "root": "human/retrieval_tasks/clapnq",
        "corpus_file": "clapnq.jsonl",
        "query_file": "clapnq_rewrite.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "fiqa": {
        "collection_name": "mt-rag-fiqa-beir-elser-512-100-20240501",
        "root": "human/retrieval_tasks/fiqa",
        "corpus_file": "fiqa.jsonl",
        "query_file": "fiqa_rewrite.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "govt": {
        "collection_name": "mt-rag-govt-elser-512-100-20240611",
        "root": "human/retrieval_tasks/govt",
        "corpus_file": "govt.jsonl",
        "query_file": "govt_rewrite.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "cloud": {
        "collection_name": "mt-rag-ibmcloud-elser-512-100-20240502",
        "root": "human/retrieval_tasks/cloud",
        "corpus_file": "cloud.jsonl",
        "query_file": "cloud_rewrite.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
}
model_name = "Qwen3"
task_name = "rewrite"

SUB_PATH = f"outputs/{model_name}_{task_name}.jsonl"
SCORED_PATH = f"outputs/{model_name}_{task_name}_score.jsonl"

sub_path = build_submission(SUB_PATH)
run_official_eval(SUB_PATH, SCORED_PATH)

In [10]:
run_official_eval(SUB_PATH, SCORED_PATH)

Running: python3 scripts/evaluation/run_retrieval_eval.py --input_file outputs/Qwen3_rewrite.jsonl --output_file outputs/Qwen3_rewrite_score.jsonl --model_name Qwen3 --task_name rewrite

collection_name: mt-rag-fiqa-beir-elser-512-100-20240501
Retriever Evaluation Aggregate Scores: {'nDCG': [0.37778, 0.32857, 0.34654, 0.39561], 'Recall': [0.15677, 0.30636, 0.3646, 0.47837], 'collection': 'mt-rag-fiqa-beir-elser-512-100-20240501', 'count': 180}
Weighted average Recall: [0.15677, 0.30636, 0.3646, 0.47837]
Weighted average nDCG: [0.37778, 0.32857, 0.34654, 0.39561]
Done, scored file: outputs/Qwen3_rewrite_score.jsonl


In [8]:
import os, json, torch
from tqdm import tqdm

task_name = "lastturn"
COLLECTIONS = {
    "clapnq": {
        "collection_name": "mt-rag-clapnq-elser-512-100-20240503",
        "root": "human/retrieval_tasks/clapnq",
        "corpus_file": "clapnq.jsonl",
        "query_file": f"clapnq_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "fiqa": {
        "collection_name": "mt-rag-fiqa-beir-elser-512-100-20240501",
        "root": "human/retrieval_tasks/fiqa",
        "corpus_file": "fiqa.jsonl",
        "query_file": f"fiqa_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "govt": {
        "collection_name": "mt-rag-govt-elser-512-100-20240611",
        "root": "human/retrieval_tasks/govt",
        "corpus_file": "govt.jsonl",
        "query_file": f"govt_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "cloud": {
        "collection_name": "mt-rag-ibmcloud-elser-512-100-20240502",
        "root": "human/retrieval_tasks/cloud",
        "corpus_file": "cloud.jsonl",
        "query_file": f"cloud_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
}
model_name = "Qwen3"

SUB_PATH = f"outputs/{model_name}_{task_name}.jsonl"
SCORED_PATH = f"outputs/{model_name}_{task_name}_score.jsonl"

sub_path = build_submission(SUB_PATH)
run_official_eval(SUB_PATH, SCORED_PATH)


Using device: cuda

========== Collection: clapnq ==========
Corpus: human/retrieval_tasks/clapnq/clapnq.jsonl
Queries: human/retrieval_tasks/clapnq/clapnq_lastturn.jsonl


Encoding:   1%|          | 11/1433 [01:00<2:09:40,  5.47s/it]
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x2ac9e0ee1990>>
Traceback (most recent call last):
  File "/home/smen/.conda/envs/mtrag/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [8]:
import os, json, torch
from tqdm import tqdm

task_name = "lastturn"
COLLECTIONS = {
    "clapnq": {
        "collection_name": "mt-rag-clapnq-elser-512-100-20240503",
        "root": "human/retrieval_tasks/clapnq",
        "corpus_file": "clapnq.jsonl",
        "query_file": f"clapnq_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "fiqa": {
        "collection_name": "mt-rag-fiqa-beir-elser-512-100-20240501",
        "root": "human/retrieval_tasks/fiqa",
        "corpus_file": "fiqa.jsonl",
        "query_file": f"fiqa_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "govt": {
        "collection_name": "mt-rag-govt-elser-512-100-20240611",
        "root": "human/retrieval_tasks/govt",
        "corpus_file": "govt.jsonl",
        "query_file": f"govt_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "cloud": {
        "collection_name": "mt-rag-ibmcloud-elser-512-100-20240502",
        "root": "human/retrieval_tasks/cloud",
        "corpus_file": "cloud.jsonl",
        "query_file": f"cloud_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
}
model_name = "bge15small"

SUB_PATH = f"outputs/{model_name}_{task_name}.jsonl"
SCORED_PATH = f"outputs/{model_name}_{task_name}_score.jsonl"

sub_path = build_submission(SUB_PATH)
run_official_eval(SUB_PATH, SCORED_PATH)


Using device: cuda

========== Collection: clapnq ==========
Corpus: human/retrieval_tasks/clapnq/clapnq.jsonl
Queries: human/retrieval_tasks/clapnq/clapnq_lastturn.jsonl


Encoding: 100%|██████████| 717/717 [02:21<00:00,  5.08it/s]


doc_emb shape: torch.Size([183408, 384])


Encoding: 100%|██████████| 1/1 [00:00<00:00, 15.53it/s]


q_emb shape: torch.Size([208, 384])


...

doc_emb shape: torch.Size([72442, 384])


Encoding: 100%|██████████| 1/1 [00:00<00:00,  5.35it/s]


q_emb shape: torch.Size([188, 384])


Building results: 100%|██████████| 188/188 [00:00<00:00, 3644.44it/s]



Total tasks: 777
Saved submission to: outputs/bge15small_lastturn.jsonl
Running: python3 scripts/evaluation/run_retrieval_eval.py --input_file outputs/bge15small_lastturn.jsonl --output_file outputs/bge15small_lastturn_score.jsonl --model_name bge15small --task_name lastturn

collection_name: mt-rag-clapnq-elser-512-100-20240503
Retriever Evaluation Aggregate Scores: {'nDCG': [0.37019, 0.34252, 0.36196, 0.41205], 'Recall': [0.15309, 0.31756, 0.38527, 0.50286], 'collection': 'mt-rag-clapnq-elser-512-100-20240503', 'count': 208}

collection_name: mt-rag-fiqa-beir-elser-512-100-20240501
Retriever Evaluation Aggregate Scores: {'nDCG': [0.21667, 0.1931, 0.20177, 0.23167], 'Recall': [0.09265, 0.17552, 0.20681, 0.27622], 'collection': 'mt-rag-fiqa-beir-elser-512-100-20240501', 'count': 180}

collection_name: mt-rag-govt-elser-512-100-20240611
Retriever Evaluation Aggregate Scores: {'nDCG': [0.22388, 0.2174, 0.2467, 0.28268], 'Recall': [0.10661, 0.20652, 0.27866, 0.36123], 'collection': 'mt-r

In [9]:
import os, json, torch
from tqdm import tqdm

task_name = "questions"
COLLECTIONS = {
    "clapnq": {
        "collection_name": "mt-rag-clapnq-elser-512-100-20240503",
        "root": "human/retrieval_tasks/clapnq",
        "corpus_file": "clapnq.jsonl",
        "query_file": f"clapnq_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "fiqa": {
        "collection_name": "mt-rag-fiqa-beir-elser-512-100-20240501",
        "root": "human/retrieval_tasks/fiqa",
        "corpus_file": "fiqa.jsonl",
        "query_file": f"fiqa_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "govt": {
        "collection_name": "mt-rag-govt-elser-512-100-20240611",
        "root": "human/retrieval_tasks/govt",
        "corpus_file": "govt.jsonl",
        "query_file": f"govt_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "cloud": {
        "collection_name": "mt-rag-ibmcloud-elser-512-100-20240502",
        "root": "human/retrieval_tasks/cloud",
        "corpus_file": "cloud.jsonl",
        "query_file": f"cloud_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
}
model_name = "bge15small"

SUB_PATH = f"outputs/{model_name}_{task_name}.jsonl"
SCORED_PATH = f"outputs/{model_name}_{task_name}_score.jsonl"
sub_path = build_submission(SUB_PATH)
run_official_eval(SUB_PATH, SCORED_PATH)


Using device: cuda

========== Collection: clapnq ==========
Corpus: human/retrieval_tasks/clapnq/clapnq.jsonl
Queries: human/retrieval_tasks/clapnq/clapnq_questions.jsonl


Encoding: 100%|██████████| 717/717 [02:21<00:00,  5.08it/s]


doc_emb shape: torch.Size([183408, 384])


Encoding: 100%|██████████| 1/1 [00:00<00:00, 15.29it/s]


q_emb shape: torch.Size([208, 384])


...

doc_emb shape: torch.Size([72442, 384])


Encoding: 100%|██████████| 1/1 [00:00<00:00,  5.28it/s]


q_emb shape: torch.Size([188, 384])


Building results: 100%|██████████| 188/188 [00:00<00:00, 2841.98it/s]



Total tasks: 777
Saved submission to: outputs/bge15small_questions.jsonl
Running: python3 scripts/evaluation/run_retrieval_eval.py --input_file outputs/bge15small_questions.jsonl --output_file outputs/bge15small_questions_score.jsonl --model_name bge15small --task_name questions

collection_name: mt-rag-govt-elser-512-100-20240611
Retriever Evaluation Aggregate Scores: {'nDCG': [0.15423, 0.1614, 0.18184, 0.21165], 'Recall': [0.07222, 0.16119, 0.20978, 0.28226], 'collection': 'mt-rag-govt-elser-512-100-20240611', 'count': 201}

collection_name: mt-rag-fiqa-beir-elser-512-100-20240501
Retriever Evaluation Aggregate Scores: {'nDCG': [0.1, 0.09567, 0.10404, 0.12082], 'Recall': [0.03981, 0.09491, 0.11435, 0.15296], 'collection': 'mt-rag-fiqa-beir-elser-512-100-20240501', 'count': 180}

collection_name: mt-rag-ibmcloud-elser-512-100-20240502
Retriever Evaluation Aggregate Scores: {'nDCG': [0.12234, 0.12076, 0.14182, 0.17836], 'Recall': [0.0516, 0.12181, 0.16968, 0.25833], 'collection': 'mt-

In [6]:
import os, json, torch
from tqdm import tqdm

task_name = "rewrite"
COLLECTIONS = {
    "clapnq": {
        "collection_name": "mt-rag-clapnq-elser-512-100-20240503",
        "root": "human/retrieval_tasks/clapnq",
        "corpus_file": "clapnq.jsonl",
        "query_file": f"clapnq_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "fiqa": {
        "collection_name": "mt-rag-fiqa-beir-elser-512-100-20240501",
        "root": "human/retrieval_tasks/fiqa",
        "corpus_file": "fiqa.jsonl",
        "query_file": f"fiqa_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "govt": {
        "collection_name": "mt-rag-govt-elser-512-100-20240611",
        "root": "human/retrieval_tasks/govt",
        "corpus_file": "govt.jsonl",
        "query_file": f"govt_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "cloud": {
        "collection_name": "mt-rag-ibmcloud-elser-512-100-20240502",
        "root": "human/retrieval_tasks/cloud",
        "corpus_file": "cloud.jsonl",
        "query_file": f"cloud_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
}
model_name = "bge15"

SUB_PATH = f"outputs/{model_name}_{task_name}.jsonl"
SCORED_PATH = f"outputs/{model_name}_{task_name}_score.jsonl"

sub_path = build_submission(SUB_PATH)
run_official_eval(SUB_PATH, SCORED_PATH)

Using device: cuda

========== Collection: clapnq ==========
Corpus: human/retrieval_tasks/clapnq/clapnq.jsonl
Queries: human/retrieval_tasks/clapnq/clapnq_rewrite.jsonl


Encoding: 100%|██████████| 717/717 [05:21<00:00,  2.23it/s]


doc_emb shape: torch.Size([183408, 768])


Encoding: 100%|██████████| 1/1 [00:00<00:00, 31.19it/s]


q_emb shape: torch.Size([208, 768])


...

doc_emb shape: torch.Size([72442, 768])


Encoding: 100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


q_emb shape: torch.Size([188, 768])


Building results: 100%|██████████| 188/188 [00:00<00:00, 3633.96it/s]



Total tasks: 777
Saved submission to: outputs/bge15_rewrite.jsonl
Running: python3 scripts/evaluation/run_retrieval_eval.py --input_file outputs/bge15_rewrite.jsonl --output_file outputs/bge15_rewrite_score.jsonl --model_name bge15 --task_name rewrite

collection_name: mt-rag-ibmcloud-elser-512-100-20240502
Retriever Evaluation Aggregate Scores: {'nDCG': [0.23936, 0.24299, 0.27315, 0.30837], 'Recall': [0.12668, 0.24628, 0.31055, 0.38999], 'collection': 'mt-rag-ibmcloud-elser-512-100-20240502', 'count': 188}

collection_name: mt-rag-clapnq-elser-512-100-20240503
Retriever Evaluation Aggregate Scores: {'nDCG': [0.42308, 0.36574, 0.40129, 0.45911], 'Recall': [0.15585, 0.32841, 0.43843, 0.57165], 'collection': 'mt-rag-clapnq-elser-512-100-20240503', 'count': 208}

collection_name: mt-rag-govt-elser-512-100-20240611
Retriever Evaluation Aggregate Scores: {'nDCG': [0.28856, 0.27288, 0.3034, 0.35744], 'Recall': [0.12927, 0.26672, 0.33701, 0.46512], 'collection': 'mt-rag-govt-elser-512-100-20

In [7]:
import os, json, torch
from tqdm import tqdm

task_name = "lastturn"
COLLECTIONS = {
    "clapnq": {
        "collection_name": "mt-rag-clapnq-elser-512-100-20240503",
        "root": "human/retrieval_tasks/clapnq",
        "corpus_file": "clapnq.jsonl",
        "query_file": f"clapnq_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "fiqa": {
        "collection_name": "mt-rag-fiqa-beir-elser-512-100-20240501",
        "root": "human/retrieval_tasks/fiqa",
        "corpus_file": "fiqa.jsonl",
        "query_file": f"fiqa_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "govt": {
        "collection_name": "mt-rag-govt-elser-512-100-20240611",
        "root": "human/retrieval_tasks/govt",
        "corpus_file": "govt.jsonl",
        "query_file": f"govt_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "cloud": {
        "collection_name": "mt-rag-ibmcloud-elser-512-100-20240502",
        "root": "human/retrieval_tasks/cloud",
        "corpus_file": "cloud.jsonl",
        "query_file": f"cloud_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
}
model_name = "bge15"

SUB_PATH = f"outputs/{model_name}_{task_name}.jsonl"
SCORED_PATH = f"outputs/{model_name}_{task_name}_score.jsonl"

sub_path = build_submission(SUB_PATH)
run_official_eval(SUB_PATH, SCORED_PATH)

Using device: cuda

========== Collection: clapnq ==========
Corpus: human/retrieval_tasks/clapnq/clapnq.jsonl
Queries: human/retrieval_tasks/clapnq/clapnq_lastturn.jsonl


Encoding: 100%|██████████| 717/717 [05:15<00:00,  2.27it/s]


doc_emb shape: torch.Size([183408, 768])


Encoding: 100%|██████████| 1/1 [00:00<00:00,  6.60it/s]


q_emb shape: torch.Size([208, 768])


...

doc_emb shape: torch.Size([72442, 768])


Encoding: 100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


q_emb shape: torch.Size([188, 768])


Building results: 100%|██████████| 188/188 [00:00<00:00, 3787.98it/s]



Total tasks: 777
Saved submission to: outputs/bge15_lastturn.jsonl
Running: python3 scripts/evaluation/run_retrieval_eval.py --input_file outputs/bge15_lastturn.jsonl --output_file outputs/bge15_lastturn_score.jsonl --model_name bge15 --task_name lastturn

collection_name: mt-rag-ibmcloud-elser-512-100-20240502
Retriever Evaluation Aggregate Scores: {'nDCG': [0.21277, 0.20314, 0.22704, 0.25412], 'Recall': [0.10496, 0.20009, 0.25603, 0.31948], 'collection': 'mt-rag-ibmcloud-elser-512-100-20240502', 'count': 188}

collection_name: mt-rag-clapnq-elser-512-100-20240503
Retriever Evaluation Aggregate Scores: {'nDCG': [0.33654, 0.29499, 0.32687, 0.38005], 'Recall': [0.12115, 0.26647, 0.35971, 0.48631], 'collection': 'mt-rag-clapnq-elser-512-100-20240503', 'count': 208}

collection_name: mt-rag-govt-elser-512-100-20240611
Retriever Evaluation Aggregate Scores: {'nDCG': [0.19403, 0.20913, 0.23521, 0.27631], 'Recall': [0.08765, 0.20954, 0.27056, 0.36388], 'collection': 'mt-rag-govt-elser-512-1

In [8]:
import os, json, torch
from tqdm import tqdm

task_name = "questions"
COLLECTIONS = {
    "clapnq": {
        "collection_name": "mt-rag-clapnq-elser-512-100-20240503",
        "root": "human/retrieval_tasks/clapnq",
        "corpus_file": "clapnq.jsonl",
        "query_file": f"clapnq_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "fiqa": {
        "collection_name": "mt-rag-fiqa-beir-elser-512-100-20240501",
        "root": "human/retrieval_tasks/fiqa",
        "corpus_file": "fiqa.jsonl",
        "query_file": f"fiqa_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "govt": {
        "collection_name": "mt-rag-govt-elser-512-100-20240611",
        "root": "human/retrieval_tasks/govt",
        "corpus_file": "govt.jsonl",
        "query_file": f"govt_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
    "cloud": {
        "collection_name": "mt-rag-ibmcloud-elser-512-100-20240502",
        "root": "human/retrieval_tasks/cloud",
        "corpus_file": "cloud.jsonl",
        "query_file": f"cloud_{task_name}.jsonl",
        "qrels_file": "qrels/dev.tsv",
    },
}
model_name = "bge15"

SUB_PATH = f"outputs/{model_name}_{task_name}.jsonl"
SCORED_PATH = f"outputs/{model_name}_{task_name}_score.jsonl"

sub_path = build_submission(SUB_PATH)
run_official_eval(SUB_PATH, SCORED_PATH)

Using device: cuda

========== Collection: clapnq ==========
Corpus: human/retrieval_tasks/clapnq/clapnq.jsonl
Queries: human/retrieval_tasks/clapnq/clapnq_questions.jsonl


Encoding: 100%|██████████| 717/717 [05:15<00:00,  2.27it/s]


doc_emb shape: torch.Size([183408, 768])


Encoding: 100%|██████████| 1/1 [00:00<00:00,  7.11it/s]


q_emb shape: torch.Size([208, 768])


...

doc_emb shape: torch.Size([72442, 768])


Encoding: 100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


q_emb shape: torch.Size([188, 768])


Building results: 100%|██████████| 188/188 [00:00<00:00, 1087.10it/s]



Total tasks: 777
Saved submission to: outputs/bge15_questions.jsonl
Running: python3 scripts/evaluation/run_retrieval_eval.py --input_file outputs/bge15_questions.jsonl --output_file outputs/bge15_questions_score.jsonl --model_name bge15 --task_name questions

collection_name: mt-rag-ibmcloud-elser-512-100-20240502
Retriever Evaluation Aggregate Scores: {'nDCG': [0.09043, 0.12217, 0.14433, 0.16774], 'Recall': [0.03785, 0.13378, 0.18227, 0.23821], 'collection': 'mt-rag-ibmcloud-elser-512-100-20240502', 'count': 188}

collection_name: mt-rag-govt-elser-512-100-20240611
Retriever Evaluation Aggregate Scores: {'nDCG': [0.14428, 0.14106, 0.16065, 0.18921], 'Recall': [0.06667, 0.14154, 0.18441, 0.25307], 'collection': 'mt-rag-govt-elser-512-100-20240611', 'count': 201}

collection_name: mt-rag-fiqa-beir-elser-512-100-20240501
Retriever Evaluation Aggregate Scores: {'nDCG': [0.1, 0.09135, 0.10274, 0.11544], 'Recall': [0.04769, 0.08889, 0.11481, 0.14602], 'collection': 'mt-rag-fiqa-beir-elser

In [9]:
# model_name = "bge15small"
# task_name = "lastturn"
# print(model_name, task_name)
# SUB_PATH = f"outputs/{model_name}_{task_name}.jsonl"
# SCORED_PATH = f"outputs/{model_name}_{task_name}_score.jsonl"

# run_official_eval(SUB_PATH, SCORED_PATH)